# Data Pre-Processing Pipeline

In [59]:
import numpy as np
import pandas as pd
import re
import warnings

# Importing necessary libraries for text processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import string

warnings.filterwarnings("ignore")

## Combine All Recipe Data

In [60]:
import yaml

def load_params():
    with open("../params.yaml") as f:
        params = yaml.safe_load(f)
    return params

params = load_params()

In [61]:
import os
import ast
data_dir = "../" + params["data_preprocessing"]["raw_data_path"]
pickle_files = []
for root, _, files in os.walk(data_dir):
    for f in files:
        if f.endswith(".pkl"):
            pickle_files.append(os.path.join(root, f))

print("Combining recipe data...")
dfs = []
for path in pickle_files:
    print(f"Loading {path} ...")
    df = pd.read_pickle(path)
    dfs.append(df)

complete_cookbook = pd.concat(dfs, axis=0)
complete_cookbook.ingredients = complete_cookbook.ingredients.apply(lambda x: ast.literal_eval(x))
complete_cookbook = complete_cookbook.reset_index(drop=True)
print(f"Combined {len(pickle_files)} files with total {len(complete_cookbook)} recipes.")
complete_cookbook.head()

Combining recipe data...
Loading ../data/raw/omnivores_cookbook_recipes.pkl ...
Loading ../data/raw/daily_dish_recipes.pkl ...
Loading ../data/raw/woks_of_life_recipes.pkl ...
Loading ../data/raw/just_one_cookbook_recipes.pkl ...
Loading ../data/raw/minimalist_baker_recipes.pkl ...
Loading ../data/raw/recipe_tin_eats_recipes.pkl ...
Loading ../data/raw/love_and_lemons_recipes.pkl ...
Loading ../data/raw/spoon_fork_bacon_recipes.pkl ...
Combined 8 files with total 8625 recipes.


,recipe_title,recipe_url,ingredients,img_url,num_steps,prep_time,cook_time,total_time,serving_size,calories,...,fiber,sugar,calcium,iron,custom_time,vitamin_c,vitamin_a,trans_fat,polyunsaturated_fat,monounsaturated_fat
0,Ginger Fried Rice,https://omnivorescookbook.com/ginger-fried-rice/,"[ground chicken, Shaoxing wine, soy sauce, sal...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,10.0,25.0,1.0,413.0,...,1.2,0.8,46.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Kung Pao Tofu (宫爆豆腐),https://omnivorescookbook.com/kung-pao-tofu/,"[firm tofu, soy sauce, maple syrup, cornstarch...",https://omnivorescookbook.com/wp-content/uploa...,8.0,20.0,20.0,40.0,1.0,302.0,...,3.0,7.3,229.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Air Fryer Chinese Roast Chicken,https://omnivorescookbook.com/air-fryer-chines...,"[whole chicken, Shaoxing wine, salt, black pep...",https://omnivorescookbook.com/wp-content/uploa...,3.0,10.0,50.0,180.0,1.0,237.0,...,0.1,1.0,14.0,1.0,120.0,NaN,NaN,NaN,NaN,NaN
3,Roast Pork Lo Mein (叉烧捞面),https://omnivorescookbook.com/pork-lo-mein/,"[fresh lo mein noodles, chicken stock, oyster ...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,15.0,30.0,1.0,457.0,...,3.9,11.1,96.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,"Sichuan Dumplings (钟水饺, Zhong Shui Jiao)",https://omnivorescookbook.com/sichuan-dumplings/,"[soy sauce, scallions, ginger, bay leaf, brown...",https://omnivorescookbook.com/wp-content/uploa...,6.0,45.0,15.0,60.0,1.0,346.0,...,1.1,4.1,9.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


## General Filtering

In [62]:
cleaned_cookbook = complete_cookbook.copy()
cleaned_cookbook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8625 entries, 0 to 8624
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   recipe_title         8625 non-null   string 
 1   recipe_url           8625 non-null   string 
 2   ingredients          8625 non-null   object 
 3   img_url              8625 non-null   string 
 4   num_steps            8625 non-null   float64
 5   prep_time            8344 non-null   float64
 6   cook_time            7046 non-null   float64
 7   total_time           7509 non-null   float64
 8   serving_size         3055 non-null   float64
 9   calories             7285 non-null   float64
 10  carbohydrates        7092 non-null   float64
 11  protein              7096 non-null   float64
 12  fat                  7056 non-null   float64
 13  saturated_fat        6875 non-null   float64
 14  cholesterol          5855 non-null   float64
 15  sodium               7014 non-null   f

### Clean up Recipe Title

In [ ]:
def clean_recipe_title(title: string) -> string:
    """
    Cleans the recipe title by removing characters that are not digits or alphabets and extra spaces.
    Also removes certain words from the title and ensure capitalization.
    """
    line = re.sub("\s*\([^()]*\)\s*", "", title) # Remove text within parentheses
    line = re.sub("[^ \-a-zA-Z0-9()]*", "", line) # Replace unwanted characters with space
    line = re.sub(r"\s*\{[^{}]*\}\s*", "", line) # Remove text within brackets
    line = re.sub("\s*\([^()]*", "", line) # Remove uncomplete parentheses

    # Remove certain words from recipe titles
    line = re.sub("(recipe|how\s*to\s*\w*\s|best|easy)", "", line, flags=re.IGNORECASE).strip()
    line = line.strip()  # Remove leading and trailing spaces
    return line.title()  # Capitalize the first letter of each word

# cleaned_cookbook["recipe_title"] = cleaned_cookbook.recipe_title.apply(clean_recipe_title)
cleaned_cookbook

,recipe_title,recipe_url,ingredients,img_url,num_steps,prep_time,cook_time,total_time,serving_size,calories,...,fiber,sugar,calcium,iron,custom_time,vitamin_c,vitamin_a,trans_fat,polyunsaturated_fat,monounsaturated_fat
0,Ginger Fried Rice,https://omnivorescookbook.com/ginger-fried-rice/,"[ground chicken, Shaoxing wine, soy sauce, sal...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,10.0,25.0,1.0,413.0,...,1.2,0.8,46.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Kung Pao Tofu,https://omnivorescookbook.com/kung-pao-tofu/,"[firm tofu, soy sauce, maple syrup, cornstarch...",https://omnivorescookbook.com/wp-content/uploa...,8.0,20.0,20.0,40.0,1.0,302.0,...,3.0,7.3,229.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Air Fryer Chinese Roast Chicken,https://omnivorescookbook.com/air-fryer-chines...,"[whole chicken, Shaoxing wine, salt, black pep...",https://omnivorescookbook.com/wp-content/uploa...,3.0,10.0,50.0,180.0,1.0,237.0,...,0.1,1.0,14.0,1.0,120.0,NaN,NaN,NaN,NaN,NaN
3,Roast Pork Lo Mein,https://omnivorescookbook.com/pork-lo-mein/,"[fresh lo mein noodles, chicken stock, oyster ...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,15.0,30.0,1.0,457.0,...,3.9,11.1,96.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Sichuan Dumplings,https://omnivorescookbook.com/sichuan-dumplings/,"[soy sauce, scallions, ginger, bay leaf, brown...",https://omnivorescookbook.com/wp-content/uploa...,6.0,45.0,15.0,60.0,1.0,346.0,...,1.1,4.1,9.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,Brown Butter Colcannon,https://www.spoonforkbacon.com/brown_butter_co...,"[extra virgin olive oil, yellow onion, bunch ...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,25.0,40.0,NaN,307.0,...,1.0,1.0,37.0,1.0,NaN,11.0,1240.0,1.00,1.0,10.0
8621,Breakfast Scramble Stuffed Avocado,https://www.spoonforkbacon.com/breakfast-scram...,"[unsalted butter, diced Canadian bacon, eggs, ...",https://www.spoonforkbacon.com/wordpress/wp-co...,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8622,Mozzarella Stuffed Chicken Parmesan Meatballs,https://www.spoonforkbacon.com/mozzarella-stuf...,"[ground chicken, oat flour, minced basil, min...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,16.0,31.0,NaN,515.0,...,3.0,1.0,387.0,4.0,NaN,2.0,660.0,1.00,3.0,8.0
8623,Potstickers,https://www.spoonforkbacon.com/pot-stickers/,"[ground pork, baby bok choy, diced, water ches...",https://www.spoonforkbacon.com/wp-content/uplo...,10.0,30.0,10.0,40.0,NaN,264.0,...,1.0,1.0,47.0,2.0,NaN,6.0,682.0,0.04,5.0,5.0


### Remove Compilation Recipe

In [64]:
# Drop compilation recipes and keep single recipes only
all_numbers_title_idx = ~pd.isna(cleaned_cookbook.recipe_title.apply(lambda x: re.match("^\d+", x.lower()))) # All titles with numbers in front 
keep_numbers_title_idx = ~pd.isna(cleaned_cookbook.recipe_title.apply(lambda x: re.match("^\d+( |-)+(ingredient(s)?|minute(s)?|hour(s)?|bowl(s)?|pot(s)?|layer(s)?)", x.lower()))) # Titles with numbers in front to keep

cleaned_cookbook = cleaned_cookbook.drop(cleaned_cookbook[all_numbers_title_idx & ~keep_numbers_title_idx].index)
cleaned_cookbook = cleaned_cookbook.reset_index(drop=True)
cleaned_cookbook

,recipe_title,recipe_url,ingredients,img_url,num_steps,prep_time,cook_time,total_time,serving_size,calories,...,fiber,sugar,calcium,iron,custom_time,vitamin_c,vitamin_a,trans_fat,polyunsaturated_fat,monounsaturated_fat
0,Ginger Fried Rice,https://omnivorescookbook.com/ginger-fried-rice/,"[ground chicken, Shaoxing wine, soy sauce, sal...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,10.0,25.0,1.0,413.0,...,1.2,0.8,46.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Kung Pao Tofu,https://omnivorescookbook.com/kung-pao-tofu/,"[firm tofu, soy sauce, maple syrup, cornstarch...",https://omnivorescookbook.com/wp-content/uploa...,8.0,20.0,20.0,40.0,1.0,302.0,...,3.0,7.3,229.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Air Fryer Chinese Roast Chicken,https://omnivorescookbook.com/air-fryer-chines...,"[whole chicken, Shaoxing wine, salt, black pep...",https://omnivorescookbook.com/wp-content/uploa...,3.0,10.0,50.0,180.0,1.0,237.0,...,0.1,1.0,14.0,1.0,120.0,NaN,NaN,NaN,NaN,NaN
3,Roast Pork Lo Mein,https://omnivorescookbook.com/pork-lo-mein/,"[fresh lo mein noodles, chicken stock, oyster ...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,15.0,30.0,1.0,457.0,...,3.9,11.1,96.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Sichuan Dumplings,https://omnivorescookbook.com/sichuan-dumplings/,"[soy sauce, scallions, ginger, bay leaf, brown...",https://omnivorescookbook.com/wp-content/uploa...,6.0,45.0,15.0,60.0,1.0,346.0,...,1.1,4.1,9.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8513,Brown Butter Colcannon,https://www.spoonforkbacon.com/brown_butter_co...,"[extra virgin olive oil, yellow onion, bunch ...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,25.0,40.0,NaN,307.0,...,1.0,1.0,37.0,1.0,NaN,11.0,1240.0,1.00,1.0,10.0
8514,Breakfast Scramble Stuffed Avocado,https://www.spoonforkbacon.com/breakfast-scram...,"[unsalted butter, diced Canadian bacon, eggs, ...",https://www.spoonforkbacon.com/wordpress/wp-co...,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8515,Mozzarella Stuffed Chicken Parmesan Meatballs,https://www.spoonforkbacon.com/mozzarella-stuf...,"[ground chicken, oat flour, minced basil, min...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,16.0,31.0,NaN,515.0,...,3.0,1.0,387.0,4.0,NaN,2.0,660.0,1.00,3.0,8.0
8516,Potstickers,https://www.spoonforkbacon.com/pot-stickers/,"[ground pork, baby bok choy, diced, water ches...",https://www.spoonforkbacon.com/wp-content/uplo...,10.0,30.0,10.0,40.0,NaN,264.0,...,1.0,1.0,47.0,2.0,NaN,6.0,682.0,0.04,5.0,5.0


### Remove non-meal recipe (sauces, dressing, store direction, etc)

In [65]:
other_recipes_idx = ~pd.isna(cleaned_cookbook.recipe_title.apply(lambda x: re.search("(sauce(s)?|dressing(s)?|store|cut|slice|clean|what)", x.lower())))

cleaned_cookbook = cleaned_cookbook.drop(cleaned_cookbook[other_recipes_idx].index)
cleaned_cookbook = cleaned_cookbook.reset_index(drop=True)
cleaned_cookbook

,recipe_title,recipe_url,ingredients,img_url,num_steps,prep_time,cook_time,total_time,serving_size,calories,...,fiber,sugar,calcium,iron,custom_time,vitamin_c,vitamin_a,trans_fat,polyunsaturated_fat,monounsaturated_fat
0,Ginger Fried Rice,https://omnivorescookbook.com/ginger-fried-rice/,"[ground chicken, Shaoxing wine, soy sauce, sal...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,10.0,25.0,1.0,413.0,...,1.2,0.8,46.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Kung Pao Tofu,https://omnivorescookbook.com/kung-pao-tofu/,"[firm tofu, soy sauce, maple syrup, cornstarch...",https://omnivorescookbook.com/wp-content/uploa...,8.0,20.0,20.0,40.0,1.0,302.0,...,3.0,7.3,229.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Air Fryer Chinese Roast Chicken,https://omnivorescookbook.com/air-fryer-chines...,"[whole chicken, Shaoxing wine, salt, black pep...",https://omnivorescookbook.com/wp-content/uploa...,3.0,10.0,50.0,180.0,1.0,237.0,...,0.1,1.0,14.0,1.0,120.0,NaN,NaN,NaN,NaN,NaN
3,Roast Pork Lo Mein,https://omnivorescookbook.com/pork-lo-mein/,"[fresh lo mein noodles, chicken stock, oyster ...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,15.0,30.0,1.0,457.0,...,3.9,11.1,96.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Sichuan Dumplings,https://omnivorescookbook.com/sichuan-dumplings/,"[soy sauce, scallions, ginger, bay leaf, brown...",https://omnivorescookbook.com/wp-content/uploa...,6.0,45.0,15.0,60.0,1.0,346.0,...,1.1,4.1,9.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7944,Brown Butter Colcannon,https://www.spoonforkbacon.com/brown_butter_co...,"[extra virgin olive oil, yellow onion, bunch ...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,25.0,40.0,NaN,307.0,...,1.0,1.0,37.0,1.0,NaN,11.0,1240.0,1.00,1.0,10.0
7945,Breakfast Scramble Stuffed Avocado,https://www.spoonforkbacon.com/breakfast-scram...,"[unsalted butter, diced Canadian bacon, eggs, ...",https://www.spoonforkbacon.com/wordpress/wp-co...,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7946,Mozzarella Stuffed Chicken Parmesan Meatballs,https://www.spoonforkbacon.com/mozzarella-stuf...,"[ground chicken, oat flour, minced basil, min...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,16.0,31.0,NaN,515.0,...,3.0,1.0,387.0,4.0,NaN,2.0,660.0,1.00,3.0,8.0
7947,Potstickers,https://www.spoonforkbacon.com/pot-stickers/,"[ground pork, baby bok choy, diced, water ches...",https://www.spoonforkbacon.com/wp-content/uplo...,10.0,30.0,10.0,40.0,NaN,264.0,...,1.0,1.0,47.0,2.0,NaN,6.0,682.0,0.04,5.0,5.0


### Remove 1-2 ingredient recipes

Most likely to be tutorial and not an actual recipe

In [66]:
less_than_two_ingredients = cleaned_cookbook[cleaned_cookbook.ingredients.apply(lambda x: len(x)) <= 2]

cleaned_cookbook = cleaned_cookbook.drop(less_than_two_ingredients.index)
cleaned_cookbook

,recipe_title,recipe_url,ingredients,img_url,num_steps,prep_time,cook_time,total_time,serving_size,calories,...,fiber,sugar,calcium,iron,custom_time,vitamin_c,vitamin_a,trans_fat,polyunsaturated_fat,monounsaturated_fat
0,Ginger Fried Rice,https://omnivorescookbook.com/ginger-fried-rice/,"[ground chicken, Shaoxing wine, soy sauce, sal...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,10.0,25.0,1.0,413.0,...,1.2,0.8,46.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Kung Pao Tofu,https://omnivorescookbook.com/kung-pao-tofu/,"[firm tofu, soy sauce, maple syrup, cornstarch...",https://omnivorescookbook.com/wp-content/uploa...,8.0,20.0,20.0,40.0,1.0,302.0,...,3.0,7.3,229.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Air Fryer Chinese Roast Chicken,https://omnivorescookbook.com/air-fryer-chines...,"[whole chicken, Shaoxing wine, salt, black pep...",https://omnivorescookbook.com/wp-content/uploa...,3.0,10.0,50.0,180.0,1.0,237.0,...,0.1,1.0,14.0,1.0,120.0,NaN,NaN,NaN,NaN,NaN
3,Roast Pork Lo Mein,https://omnivorescookbook.com/pork-lo-mein/,"[fresh lo mein noodles, chicken stock, oyster ...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,15.0,30.0,1.0,457.0,...,3.9,11.1,96.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Sichuan Dumplings,https://omnivorescookbook.com/sichuan-dumplings/,"[soy sauce, scallions, ginger, bay leaf, brown...",https://omnivorescookbook.com/wp-content/uploa...,6.0,45.0,15.0,60.0,1.0,346.0,...,1.1,4.1,9.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7944,Brown Butter Colcannon,https://www.spoonforkbacon.com/brown_butter_co...,"[extra virgin olive oil, yellow onion, bunch ...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,25.0,40.0,NaN,307.0,...,1.0,1.0,37.0,1.0,NaN,11.0,1240.0,1.00,1.0,10.0
7945,Breakfast Scramble Stuffed Avocado,https://www.spoonforkbacon.com/breakfast-scram...,"[unsalted butter, diced Canadian bacon, eggs, ...",https://www.spoonforkbacon.com/wordpress/wp-co...,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7946,Mozzarella Stuffed Chicken Parmesan Meatballs,https://www.spoonforkbacon.com/mozzarella-stuf...,"[ground chicken, oat flour, minced basil, min...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,16.0,31.0,NaN,515.0,...,3.0,1.0,387.0,4.0,NaN,2.0,660.0,1.00,3.0,8.0
7947,Potstickers,https://www.spoonforkbacon.com/pot-stickers/,"[ground pork, baby bok choy, diced, water ches...",https://www.spoonforkbacon.com/wp-content/uplo...,10.0,30.0,10.0,40.0,NaN,264.0,...,1.0,1.0,47.0,2.0,NaN,6.0,682.0,0.04,5.0,5.0


## Handling NA Values

In [67]:
cleaned_cookbook.isna().sum()

recipe_title              0
recipe_url                0
ingredients               0
img_url                   0
num_steps                 0
prep_time               225
cook_time              1302
total_time              978
serving_size           4992
calories               1101
carbohydrates          1276
protein                1270
fat                    1300
saturated_fat          1453
cholesterol            2328
sodium                 1351
potassium              1971
fiber                  1495
sugar                  1367
calcium                2073
iron                   2093
custom_time            6794
vitamin_c              2952
vitamin_a              2730
trans_fat              4669
polyunsaturated_fat    4733
monounsaturated_fat    4789
dtype: int64

### Time

Fill NA total time based on other times in case it wasn't catched when scraping. Otherwise, drop and fill the rest time variables with 0s

In [68]:
cleaned_cookbook.total_time = cleaned_cookbook.total_time.fillna(cleaned_cookbook.prep_time + cleaned_cookbook.cook_time + cleaned_cookbook.custom_time)
cleaned_cookbook = cleaned_cookbook.dropna(subset="total_time")
cleaned_cookbook

,recipe_title,recipe_url,ingredients,img_url,num_steps,prep_time,cook_time,total_time,serving_size,calories,...,fiber,sugar,calcium,iron,custom_time,vitamin_c,vitamin_a,trans_fat,polyunsaturated_fat,monounsaturated_fat
0,Ginger Fried Rice,https://omnivorescookbook.com/ginger-fried-rice/,"[ground chicken, Shaoxing wine, soy sauce, sal...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,10.0,25.0,1.0,413.0,...,1.2,0.8,46.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Kung Pao Tofu,https://omnivorescookbook.com/kung-pao-tofu/,"[firm tofu, soy sauce, maple syrup, cornstarch...",https://omnivorescookbook.com/wp-content/uploa...,8.0,20.0,20.0,40.0,1.0,302.0,...,3.0,7.3,229.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Air Fryer Chinese Roast Chicken,https://omnivorescookbook.com/air-fryer-chines...,"[whole chicken, Shaoxing wine, salt, black pep...",https://omnivorescookbook.com/wp-content/uploa...,3.0,10.0,50.0,180.0,1.0,237.0,...,0.1,1.0,14.0,1.0,120.0,NaN,NaN,NaN,NaN,NaN
3,Roast Pork Lo Mein,https://omnivorescookbook.com/pork-lo-mein/,"[fresh lo mein noodles, chicken stock, oyster ...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,15.0,30.0,1.0,457.0,...,3.9,11.1,96.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Sichuan Dumplings,https://omnivorescookbook.com/sichuan-dumplings/,"[soy sauce, scallions, ginger, bay leaf, brown...",https://omnivorescookbook.com/wp-content/uploa...,6.0,45.0,15.0,60.0,1.0,346.0,...,1.1,4.1,9.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7943,Mm Cookies,https://www.spoonforkbacon.com/mm-cookies/,"[all purpose flour, baking soda, baking powder...",https://www.spoonforkbacon.com/wp-content/uplo...,10.0,10.0,12.0,82.0,NaN,248.0,...,1.0,23.0,36.0,1.0,60.0,1.0,290.0,1.00,1.0,2.0
7944,Brown Butter Colcannon,https://www.spoonforkbacon.com/brown_butter_co...,"[extra virgin olive oil, yellow onion, bunch ...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,25.0,40.0,NaN,307.0,...,1.0,1.0,37.0,1.0,NaN,11.0,1240.0,1.00,1.0,10.0
7946,Mozzarella Stuffed Chicken Parmesan Meatballs,https://www.spoonforkbacon.com/mozzarella-stuf...,"[ground chicken, oat flour, minced basil, min...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,16.0,31.0,NaN,515.0,...,3.0,1.0,387.0,4.0,NaN,2.0,660.0,1.00,3.0,8.0
7947,Potstickers,https://www.spoonforkbacon.com/pot-stickers/,"[ground pork, baby bok choy, diced, water ches...",https://www.spoonforkbacon.com/wp-content/uplo...,10.0,30.0,10.0,40.0,NaN,264.0,...,1.0,1.0,47.0,2.0,NaN,6.0,682.0,0.04,5.0,5.0


In [69]:
cleaned_cookbook[["prep_time", "cook_time", "custom_time"]] = cleaned_cookbook[["prep_time", "cook_time", "custom_time"]].fillna(0)

In [70]:
cleaned_cookbook[["total_time", "prep_time", "cook_time", "custom_time"]].isna().sum()

total_time     0
prep_time      0
cook_time      0
custom_time    0
dtype: int64

### Macros and Micros

Fill NAs with 0s

In [71]:
fill_idx = cleaned_cookbook.columns[8:-1]
cleaned_cookbook[fill_idx].isna().sum()

serving_size           4282
calories                849
carbohydrates          1019
protein                1012
fat                    1041
saturated_fat          1185
cholesterol            1983
sodium                 1094
potassium              1705
fiber                  1221
sugar                  1107
calcium                1801
iron                   1826
custom_time               0
vitamin_c              2626
vitamin_a              2433
trans_fat              4039
polyunsaturated_fat    4102
dtype: int64

In [72]:
cleaned_cookbook[fill_idx] = cleaned_cookbook[fill_idx].fillna(0)

In [73]:
cleaned_cookbook[fill_idx].isna().sum()

serving_size           0
calories               0
carbohydrates          0
protein                0
fat                    0
saturated_fat          0
cholesterol            0
sodium                 0
potassium              0
fiber                  0
sugar                  0
calcium                0
iron                   0
custom_time            0
vitamin_c              0
vitamin_a              0
trans_fat              0
polyunsaturated_fat    0
dtype: int64

## Handling Duplicated Values

In [74]:
cleaned_cookbook.duplicated(subset=["recipe_title"]).sum()

367

In [75]:
cleaned_cookbook = cleaned_cookbook.drop_duplicates(subset=["recipe_title"])
cleaned_cookbook

,recipe_title,recipe_url,ingredients,img_url,num_steps,prep_time,cook_time,total_time,serving_size,calories,...,fiber,sugar,calcium,iron,custom_time,vitamin_c,vitamin_a,trans_fat,polyunsaturated_fat,monounsaturated_fat
0,Ginger Fried Rice,https://omnivorescookbook.com/ginger-fried-rice/,"[ground chicken, Shaoxing wine, soy sauce, sal...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,10.0,25.0,1.0,413.0,...,1.2,0.8,46.0,3.0,0.0,0.0,0.0,0.00,0.0,NaN
1,Kung Pao Tofu,https://omnivorescookbook.com/kung-pao-tofu/,"[firm tofu, soy sauce, maple syrup, cornstarch...",https://omnivorescookbook.com/wp-content/uploa...,8.0,20.0,20.0,40.0,1.0,302.0,...,3.0,7.3,229.0,3.0,0.0,0.0,0.0,0.00,0.0,NaN
2,Air Fryer Chinese Roast Chicken,https://omnivorescookbook.com/air-fryer-chines...,"[whole chicken, Shaoxing wine, salt, black pep...",https://omnivorescookbook.com/wp-content/uploa...,3.0,10.0,50.0,180.0,1.0,237.0,...,0.1,1.0,14.0,1.0,120.0,0.0,0.0,0.00,0.0,NaN
3,Roast Pork Lo Mein,https://omnivorescookbook.com/pork-lo-mein/,"[fresh lo mein noodles, chicken stock, oyster ...",https://omnivorescookbook.com/wp-content/uploa...,7.0,15.0,15.0,30.0,1.0,457.0,...,3.9,11.1,96.0,4.0,0.0,0.0,0.0,0.00,0.0,NaN
4,Sichuan Dumplings,https://omnivorescookbook.com/sichuan-dumplings/,"[soy sauce, scallions, ginger, bay leaf, brown...",https://omnivorescookbook.com/wp-content/uploa...,6.0,45.0,15.0,60.0,1.0,346.0,...,1.1,4.1,9.0,1.0,0.0,0.0,0.0,0.00,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7943,Mm Cookies,https://www.spoonforkbacon.com/mm-cookies/,"[all purpose flour, baking soda, baking powder...",https://www.spoonforkbacon.com/wp-content/uplo...,10.0,10.0,12.0,82.0,0.0,248.0,...,1.0,23.0,36.0,1.0,60.0,1.0,290.0,1.00,1.0,2.0
7944,Brown Butter Colcannon,https://www.spoonforkbacon.com/brown_butter_co...,"[extra virgin olive oil, yellow onion, bunch ...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,25.0,40.0,0.0,307.0,...,1.0,1.0,37.0,1.0,0.0,11.0,1240.0,1.00,1.0,10.0
7946,Mozzarella Stuffed Chicken Parmesan Meatballs,https://www.spoonforkbacon.com/mozzarella-stuf...,"[ground chicken, oat flour, minced basil, min...",https://www.spoonforkbacon.com/wp-content/uplo...,6.0,15.0,16.0,31.0,0.0,515.0,...,3.0,1.0,387.0,4.0,0.0,2.0,660.0,1.00,3.0,8.0
7947,Potstickers,https://www.spoonforkbacon.com/pot-stickers/,"[ground pork, baby bok choy, diced, water ches...",https://www.spoonforkbacon.com/wp-content/uplo...,10.0,30.0,10.0,40.0,0.0,264.0,...,1.0,1.0,47.0,2.0,0.0,6.0,682.0,0.04,5.0,5.0


## Tokenizing, Lemmatizing, and Removing General Stop Words for Ingredients

Clean the ingredients using multiple NLP methods and filtering general stop words

Step-by-step:
1. Normalization
2. Surface-level Filtering (numbers, punctuations, brands, etc)
3. Tokenization
4. Lemmatization
5. Stop-Words Filtering

In [76]:
# Identify stop words specific to recipes
PREP_WORDS = ["thinly", "finely", "softened", "crushed", "skinned", "whole", "chopped", "minced"]

TEMP_WORDS = ["hot", "warm", "cold", "room", "chilled", "frozen", "cool", "boiling", "temperature"]

TASTE_WORDS = ["sweet", "sour", "bitter", "salty", "umami"]

QUALITY_WORDS = ["fresh", "freshly", "organic", "good", "quality", "extra", "premium", "best", "finest", "high", "filtered", "light", "sashimi-grade", "pure", "fine"]

SIZE_WORDS = ["large", "mini", "small", "medium", "jumbo", "extra", "extra-large", "extra-small", "sliced", "halved", "diced", "cubed", "peeled", "grated", "shredded", "quartered", "clove", "short-grain", "heaping"]

STATE_WORDS = ["powder", "sauce", "boneless", "skinless", "skin", "cooked", "ripe", "coarse", "coarsely", "filet", "fillet", "fillets", "canned",
                "homemade", "used", "dried", "reserved", "packet", "ground", "uncooked", "pasteurized", "bonein", "skinon", "preserved", "raw", "cooking"]

OTHER_WORDS = ["quality", "extra", "virgin", "extravirgin", "long", "high", "japanese", "combination", "diamond", "kosher", "stalk", "juice", "sea", "taste", "crystal", "choice"]

UNITS = ["cup", "cups", "tbsp", "tablespoon", "tsp", "teaspoon", "g", "kg", "oz", "ml", "l", "lb", "pound", "inch", "cm", "m", "handful", "bulb", "dollop", "pinch"]

EXTRA_WORDS = PREP_WORDS + TEMP_WORDS + TASTE_WORDS + QUALITY_WORDS + SIZE_WORDS + STATE_WORDS + OTHER_WORDS + UNITS

In [77]:
def clean_ingredients(ingredients: list) -> list:
    """
    A single ingredient may consist of several words, thus
    it takes each string of ingredient and process it using multiple NLP methods into a clean string.
    This function also splits ingredients that are combined by commas and "and" and processes them separately.
    """
    # Initialize functions for cleaning
    translator = str.maketrans('', '', '!"#$%&\'()*+,.:;<=>?@[\\]^_`{|}~') # Punctuations remover (exclue slash "/" and strip "-")
    lemmatizer = WordNetLemmatizer() # Word lemmatizer
    
    clean_ingredients = []
    for ingredient in ingredients:
        # Split by commas and "and" to handle combined ingredients
        split_ingredients = [ing.strip() for ing in re.split(r'\s+(and|&|,)\s+', ingredient, flags=re.IGNORECASE) if ing.strip()]
        split_ingredients = list(dict.fromkeys(split_ingredients))  # Remove duplicates from split ingredients

        # Process each split ingredient separately
        for sub_ingredient in split_ingredients:
            # Normalization
            line = sub_ingredient.lower() # Make sure string are lowercase
            line = re.sub("\s*\([^()]*\)\s*", "", line) # Remove text between parentheses
            
            # Surface level cleaning
            line = re.sub("(\s+(or|/){1}\s+)", "/", line) # Remove ingredient substitutes
            # line = re.sub("(diamond crystal|premium-quality)", "", line) # Remove brands
            # line = re.sub("(\w*\d\w*|½|¼|¾)", "", line) # Remove numbers

            # Remove punctuations
            line = line.translate(translator)

            # In-depth cleaning
            line_tokenized = word_tokenize(line) # Tokenize the ingredient first
            line_lemmatized = [lemmatizer.lemmatize(ing) for ing in line_tokenized] # Then lemmatize the ingredient
            line_split = [ing for ing in line_lemmatized if ing not in stopwords.words("english") + EXTRA_WORDS + [""]] # Remove stop words
            
            if line_split:  # Only add if there are remaining words after cleaning
                line_split = list(dict.fromkeys(line_split)) # Remove duplicates from the words in a single ingredient
                line = "_".join(line_split) # Rejoin the ingredient as a single string
                clean_ingredients.append(line) # Add back to the ingredients list
        
    clean_ingredients = list(dict.fromkeys(clean_ingredients)) # Remove duplicates
    
    return clean_ingredients

# Clean ingredients namings
cleaned_cookbook.ingredients = cleaned_cookbook.ingredients.apply(clean_ingredients)

In [419]:
from nltk.probability import FreqDist

f_dist = FreqDist(i for i in cleaned_cookbook.ingredients.explode())

f_dist.most_common(100)

[('salt', 3831),
 ('garlic', 2151),
 ('water', 1651),
 ('black_pepper', 1390),
 ('sugar', 1117),
 ('olive_oil', 1014),
 ('egg', 997),
 ('soy', 915),
 ('onion', 752),
 ('ginger', 742),
 ('vanilla_extract', 691),
 ('unsalted_butter', 633),
 ('lemon', 624),
 ('carrot', 518),
 ('all-purpose_flour', 488),
 ('baking', 477),
 ('maple_syrup', 471),
 ('scallion', 460),
 ('sesame_oil', 454),
 ('milk', 430),
 ('parsley', 423),
 ('pepper', 403),
 ('cinnamon', 394),
 ('white_pepper', 387),
 ('oil', 365),
 ('neutral_oil', 361),
 ('cilantro', 350),
 ('shaoxing_wine', 348),
 ('cornstarch', 345),
 ('vegetable_oil', 330),
 ('lime', 320),
 ('extra-virgin_olive_oil', 308),
 ('baking_soda', 303),
 ('cumin', 298),
 ('tomato', 291),
 ('brown_sugar', 271),
 ('granulated_sugar', 266),
 ('red_onion', 262),
 ('sake', 241),
 ('oyster', 240),
 ('mirin', 239),
 ('dark_soy', 232),
 ('avocado', 220),
 ('red_pepper_flake', 219),
 ('butter', 208),
 ('honey', 205),
 ('cream', 190),
 ('potato', 187),
 ('rice_vinegar', 17

## Save Cleaned Data

### Type Check

In [423]:
cleaned_cookbook.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5750 entries, 0 to 6702
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   recipe_title         5750 non-null   object 
 1   recipe_url           5750 non-null   string 
 2   ingredients          5750 non-null   object 
 3   num_steps            5750 non-null   float64
 4   total_time           5750 non-null   float64
 5   prep_time            5750 non-null   float64
 6   cook_time            5750 non-null   float64
 7   custom_time          5750 non-null   float64
 8   calories             5750 non-null   float64
 9   carbohydrates        5750 non-null   float64
 10  protein              5750 non-null   float64
 11  fat                  5750 non-null   float64
 12  saturated_fat        5750 non-null   float64
 13  polyunsaturated_fat  5750 non-null   float64
 14  monounsaturated_fat  5750 non-null   float64
 15  trans_fat            5750 non-null   float6

In [424]:
# Convert recipe_title to string
cleaned_cookbook.recipe_title = cleaned_cookbook.recipe_title.astype(str)

### Save

In [ ]:
cleaned_cookbook = cleaned_cookbook.reset_index(drop=True)
cleaned_cookbook.to_pickle("processed_cookbook.pkl", protocol=4)

## Sanity Check

In [ ]:
pd.read_pickle("processed_cookbook.pkl")

,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,carbohydrates,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
0,Tonkotsu Ramen,https://www.justonecookbook.com/easy-tonkotsu-...,"[pork_leg_bone, pork_hock, water, garlic, ging...",35.0,380.0,60.0,130.0,60.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.justonecookbook.com/wp-content/upl...
1,Pan-Fried Curry Chicken,https://www.justonecookbook.com/pan-fried-curr...,"[chicken_tender, salt, black_pepper, kewpie_ma...",15.0,40.0,10.0,10.0,0.0,290.0,10.0,...,989.0,441.0,0.5,4.0,46.0,1.0,11.0,1.0,0.0,https://www.justonecookbook.com/wp-content/upl...
2,Homemade Udon Noodles,https://www.justonecookbook.com/udon-noodles/,"[allpurpose_flour, water, salt, potato_starch]",36.0,120.0,60.0,0.0,150.0,361.0,76.0,...,198.0,106.0,3.0,1.0,0.0,0.0,21.0,5.0,0.0,https://www.justonecookbook.com/wp-content/upl...
3,Tomato Egg Vermicelli Soup,https://www.justonecookbook.com/tomato-egg-ver...,"[tomato, green_scallion, egg, chicken_broth, s...",10.0,30.0,5.0,10.0,0.0,123.0,16.0,...,299.0,215.0,2.0,3.0,737.0,11.0,79.0,2.0,0.0,https://www.justonecookbook.com/wp-content/upl...
4,Butter Ponzu Beef,https://www.justonecookbook.com/butter-ponzu-b...,"[beef, garlic, komatsuna, maitake_mushroom, ne...",13.0,40.0,10.0,10.0,0.0,386.0,8.0,...,751.0,684.0,2.0,1.0,194.0,17.0,90.0,3.0,0.0,https://www.justonecookbook.com/wp-content/upl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,Scallion Ginger Shrimp,https://thewoksoflife.com/scallion-ginger-shri...,"[shrimp, scallion, ginger, peanut_oil, shaoxin...",4.0,15.0,10.0,5.0,0.0,191.0,2.0,...,1043.0,145.0,1.0,1.0,120.0,7.2,173.0,2.6,0.0,https://thewoksoflife.com/wp-content/uploads/2...
6266,Classic Peanut Butter Cake,https://thewoksoflife.com/classic-peanut-butte...,"[allpurpose_flour, baking, baking_soda, salt, ...",4.0,60.0,30.0,30.0,0.0,517.0,53.0,...,418.0,346.0,2.0,28.0,225.0,0.0,97.0,2.2,0.0,https://thewoksoflife.com/wp-content/uploads/2...
6267,Frozen White Peach Mango Margaritas,https://thewoksoflife.com/frozen-white-peach-m...,"[mango, peach, lime, simple_syrup, tequila, tr...",3.0,10.0,10.0,0.0,0.0,206.0,30.0,...,593.0,234.0,2.0,28.0,691.0,23.0,9.0,1.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
6268,Cantonese Chicken Salted Fish Fried Rice,https://thewoksoflife.com/cantonese-chicken-sa...,"[oil, chicken_breast, onion, chinese_saltcured...",3.0,30.0,20.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...


In [ ]:
complete_cookbook[complete_cookbook.recipe_url == "https://natashaskitchen.com/cucumber-radish-salad-recipe/"]

,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,carbohydrates,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
4337,Cucumber Radish Salad Recipe (VIDEO),https://natashaskitchen.com/cucumber-radish-sa...,"[English cucumber, radishes, chives or green o...",2.0,10.0,10.0,NaN,NaN,99.0,4.0,...,479.0,232.0,NaN,2.0,565.0,8.3,69.0,0.4,NaN,https://natashaskitchen.com/wp-content/uploads...


In [ ]:
cleaned_cookbook[cleaned_cookbook.recipe_url == "https://natashaskitchen.com/cucumber-radish-salad-recipe/"]

,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,carbohydrates,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
3958,Cucumber Radish Salad,https://natashaskitchen.com/cucumber-radish-sa...,"[english_cucumber, radish, chive_onion, sour_c...",2.0,10.0,10.0,0.0,0.0,99.0,4.0,...,479.0,232.0,0.0,2.0,565.0,8.3,69.0,0.4,0.0,https://natashaskitchen.com/wp-content/uploads...
